In [24]:
import os
import numpy as np
import pandas as pd
import os
import time
import torch

In [2]:
train_series = pd.read_csv('./train_series_descriptions.csv')
train = pd.read_csv('./train.csv')
train_label = pd.read_csv('./train_label_coordinates.csv')
sample = pd.read_csv('./sample_submission.csv')
test_series = pd.read_csv('./test_series_descriptions.csv')

In [3]:
train.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [4]:
train.columns

Index(['study_id', 'spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_stenosis_l3_l4',
 

In [5]:
train_label.head()

,study_id,series_id,instance_number,condition,level,x,y
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602


In [6]:
train_series.head()

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1


In [7]:
test_series.head()

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [8]:
sample.head(100)

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.333333,0.333333,0.333333
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.333333,0.333333,0.333333
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.333333,0.333333,0.333333
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.333333,0.333333,0.333333
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.333333,0.333333,0.333333
5,44036939_left_subarticular_stenosis_l1_l2,0.333333,0.333333,0.333333
6,44036939_left_subarticular_stenosis_l2_l3,0.333333,0.333333,0.333333
7,44036939_left_subarticular_stenosis_l3_l4,0.333333,0.333333,0.333333
8,44036939_left_subarticular_stenosis_l4_l5,0.333333,0.333333,0.333333
9,44036939_left_subarticular_stenosis_l5_s1,0.333333,0.333333,0.333333


In [9]:
df_train_step_1 = pd.merge(left=train, right=train_label, how='left', on='study_id').reset_index(drop=True)
df_train_step_1.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_id,instance_number,condition,level,x,y
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L5/S1,353.415929,483.964602


In [10]:
mapping= {
    'normal/mild': 1,
    'moderate': 2,
    'severe': 3
}

In [11]:
def standardize_and_map(column, mapping):
    if column.dtype == 'object':  # Check if the column contains string values
        return column.str.lower().str.strip().map(mapping)

In [12]:
df_train = pd.merge(left=df_train_step_1, right=train_series, how='left', on=['study_id', 'series_id']).reset_index(drop=True)
df_train.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_id,instance_number,condition,level,x,y,series_description
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,702807833.0,8.0,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR


In [13]:
for column in df_train.columns:
    if column != 'condition' and column !='level' and column !='series_description' and column!='study_id' and column!='x' and column!='y':
        df_train[column] = standardize_and_map(df_train[column], mapping)

In [14]:
df_train.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_id,instance_number,condition,level,x,y,series_description
0,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,None,None,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,None,None,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,None,None,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,None,None,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,None,None,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR


In [15]:
part_1 = os.listdir('train_images')
part_1 = list(filter(lambda x: x.find('.DS') == -1, part_1))

In [16]:
p1 = [(x, f"train_images/{x}") for x in part_1]

In [17]:
print(p1)

[('100206310', 'train_images/100206310'), ('1002894806', 'train_images/1002894806'), ('1004726367', 'train_images/1004726367'), ('1008446160', 'train_images/1008446160'), ('1009445512', 'train_images/1009445512'), ('1009905322', 'train_images/1009905322'), ('1012375618', 'train_images/1012375618'), ('1013589491', 'train_images/1013589491'), ('1013791258', 'train_images/1013791258'), ('1018005303', 'train_images/1018005303'), ('1019430579', 'train_images/1019430579'), ('1020394063', 'train_images/1020394063'), ('1025265129', 'train_images/1025265129'), ('1028684462', 'train_images/1028684462'), ('1028909382', 'train_images/1028909382'), ('1035170868', 'train_images/1035170868'), ('1036203708', 'train_images/1036203708'), ('1038453736', 'train_images/1038453736'), ('1039182563', 'train_images/1039182563'), ('1040921274', 'train_images/1040921274'), ('1047914296', 'train_images/1047914296'), ('1050200728', 'train_images/1050200728'), ('1051198661', 'train_images/1051198661'), ('1051595826

In [18]:
meta_obj = { p[0]: { 'folder_path': p[1], 
                    'SeriesInstanceUIDs': [] 
                   } 
            for p in p1 }

In [19]:
meta_obj

{'100206310': {'folder_path': 'train_images/100206310',
  'SeriesInstanceUIDs': []},
 '1002894806': {'folder_path': 'train_images/1002894806',
  'SeriesInstanceUIDs': []},
 '1004726367': {'folder_path': 'train_images/1004726367',
  'SeriesInstanceUIDs': []},
 '1008446160': {'folder_path': 'train_images/1008446160',
  'SeriesInstanceUIDs': []},
 '1009445512': {'folder_path': 'train_images/1009445512',
  'SeriesInstanceUIDs': []},
 '1009905322': {'folder_path': 'train_images/1009905322',
  'SeriesInstanceUIDs': []},
 '1012375618': {'folder_path': 'train_images/1012375618',
  'SeriesInstanceUIDs': []},
 '1013589491': {'folder_path': 'train_images/1013589491',
  'SeriesInstanceUIDs': []},
 '1013791258': {'folder_path': 'train_images/1013791258',
  'SeriesInstanceUIDs': []},
 '1018005303': {'folder_path': 'train_images/1018005303',
  'SeriesInstanceUIDs': []},
 '1019430579': {'folder_path': 'train_images/1019430579',
  'SeriesInstanceUIDs': []},
 '1020394063': {'folder_path': 'train_images/

In [20]:
for m in meta_obj:
    meta_obj[m]['SeriesInstanceUIDs'] = list(
        filter(lambda x: x.find('.DS') == -1, 
               os.listdir(meta_obj[m]['folder_path'])
              )
    )

In [21]:
import pydicom
import cv2
def resize_image(image_path, target_size=(224, 224)):
    dicom = pydicom.dcmread(image_path)

    # Convert the DCM image to a numpy array
    image_array = dicom.pixel_array
    # Resize the image using OpenCV
    resized_image = cv2.resize(image_array, (224, 224))
    # Store the resized image in a numpy array
    resized_image_array = np.array(resized_image)

    return resized_image_array

In [22]:
images=[]
# Select only the columns with dtype 'object'
columns=['spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_stenosis_l3_l4',
       'right_subarticular_stenosis_l4_l5',
       'right_subarticular_stenosis_l5_s1']
labels=[[]]
for study_id in part_1:
    study_id_dir = os.listdir(f'./train_images/{study_id}')
    for series_id in study_id_dir:
        series_id_dir = os.listdir(f'./train_images/{study_id}/{series_id}')
        series_id_instances = train_label
        for instance in series_id_dir:
            image_path=f'./train_images/{study_id}/{series_id}/{instance}'
            resized_image = resize_image(image_path)
            images.append(resized_image)
            filtered_df = df_train[(df_train['study_id']==study_id) & (df_train['series_id']==series_id) & (df_train['instance_number']==instance)][columns]
            if not filtered_df.empty:
                labels.append(filtered_df.values[0])
            

In [43]:
labels

[[]]

In [29]:
# # Save the arrays
# np.save('./saved_arrays/images.npy', images)
# np.save('./saved_arrays/labels.npy', labels)

# Define the size of each batch
batch_size = 10000

# Save the arrays in batches
for i in range(0, len(images), batch_size):
    np.save(f'./saved_arrays/images_batch_{i}.npy', images[i:i+batch_size])
    np.save(f'./saved_arrays/labels_batch_{i}.npy', labels[i:i+batch_size])



In [30]:
len(images)

147218

In [32]:
# Load the arrays back in batches
loaded_images = []
loaded_labels = []
for i in range(0, 10, batch_size):
    loaded_images.append(np.load(f'./saved_arrays/images_batch_{i}.npy'))
    loaded_labels.append(np.load(f'./saved_arrays/labels_batch_{i}.npy'))

In [41]:
test = np.array(loaded_labels)
test.shape

(1, 1, 0)

In [25]:
from sklearn.model_selection import train_test_split
images = np.array(images)
labels = np.array(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 27.5 GiB for an array with shape (147218, 224, 224) and data type int32

In [ ]:
from keras.utils import to_categorical
y_train_severity = to_categorical(y_train[:, 1], num_classes=3)
y_train_condition = to_categorical(y_train[:, 2], num_classes=3)
y_train_level = to_categorical(y_train[:, 3], num_classes=3)

y_test_severity = to_categorical(y_test[:, 1], num_classes=3)
y_test_condition = to_categorical(y_test[:, 2], num_classes=3)
y_test_level = to_categorical(y_test[:, 3], num_classes=3)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_img = Input(shape=(224, 224, 3))

# Convolutional layers
x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Fully connected layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layers
severity_output = Dense(3, activation='softmax', name='severity')(x)
condition_output = Dense(3, activation='softmax', name='condition')(x)
level_output = Dense(3, activation='softmax', name='level')(x)

# Model
model = Model(inputs=input_img, outputs=[severity_output, condition_output, level_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'severity': 'categorical_crossentropy', 'condition': 'categorical_crossentropy', 'level': 'categorical_crossentropy'},
              metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, 
                    {'severity': y_train_severity, 'condition': y_train_condition, 'level': y_train_level},
                    epochs=25,
                    batch_size=32,
                    validation_split=0.2)

In [ ]:
loss, severity_acc, condition_acc, level_acc = model.evaluate(X_test, 
                                                             {'severity': y_test_severity, 'condition': y_test_condition, 'level': y_test_level})

print(f'Severity Accuracy: {severity_acc}')
print(f'Condition Accuracy: {condition_acc}')
print(f'Level Accuracy: {level_acc}')


In [ ]:
predictions = model.predict(X_test)

# Convert predictions from one-hot encoding to labels
predicted_severity = np.argmax(predictions[0], axis=1)
predicted_condition = np.argmax(predictions[1], axis=1)
predicted_level = np.argmax(predictions[2], axis=1)
